In [306]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [302]:
embeddings_dict = {}
wordslist = []
wordVectors = np.zeros((1193514, 25), dtype='float32')
id_dict = {}
sentences = []

In [348]:
embeddings_index = {}

## Read stanford glove (25d) (try out with multiple dimensions after )

In [349]:
index = 0
with open("./stanfordglove/glove.twitter.27B.25d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        wordslist.append(word)
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector
        if(vector.shape == (25,)):
            wordVectors[index] = vector
            embeddings_index[word]= vector
        id_dict[word]= index
        index+=1

## read in tweets

In [307]:
pos_train = []
pos_sentences = []
with open('train_pos.txt') as file:
    for line in file:
        sentences.append(line)
        sentence = np.zeros(60, dtype='int32')
        count = 0
        for val in line.strip().split():
            idx = id_dict.get(val, -1)
            if(idx != -1):
                sentence[count] = idx 
                count+=1
        pos_train.append(np.array(sentence))

In [375]:
pos_train_arr = np.array(pos_train)



In [309]:
neg_train = []
with open('train_neg.txt') as file:
    for line in file:
        sentences.append(line)
        sentence = np.zeros(60, dtype='int32')
        count = 0
        for val in line.strip().split():
            idx = id_dict.get(val, -1)
            if(idx != -1):
                sentence[count] = idx
                count+=1
        neg_train.append(np.array(sentence))

In [374]:
neg_train_arr= np.array(neg_train)


In [311]:
poslabels = np.repeat(1,pos_train_arr.shape[0])
neglabels = np.repeat(0,neg_train_arr.shape[0])

In [312]:
labels = np.append(poslabels,neglabels)
training_set = np.concatenate((pos_train_arr, neg_train_arr))

In [313]:
ids = training_set

In [314]:
training_set.shape[1]

60

## put data into dataframe 

In [316]:
data_new = pd.DataFrame({'text':sentences,'sentiment':labels})

## standarize  

In [317]:
#data = data[data.sentiment != "Neutral"]
data_new['text'] = data_new['text'].apply(lambda x: x.lower())
data_new['text'] = data_new['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data_new[ data_new['sentiment'] == 1].size)
print(data_new[ data_new['sentiment'] == 0].size)

for idx,row in data_new.iterrows():
    row[0] = row[0].replace('rt',' ')
    


200000
200000


In [ ]:
#how many unique words ?

In [318]:
s = {}
for sentence in data_new['text'].values:
    for word in sentence.split():
        s[word] = 1

In [319]:
len(s) 

107090

In [320]:
max_fatures = len(s) 
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data_new['text'].values)
X = tokenizer.texts_to_sequences(data_new['text'].values)
X = pad_sequences(X)

In [352]:
print('Found %s word vectors.' % len(embeddings_index))
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 1193513 word vectors.
Found 106936 unique tokens.


In [ ]:
#build embedding matrix from data to glove embeddings 

In [355]:
num_words = min(max_fatures, len(word_index)) + 1
print(num_words)

embedding_dim = 25

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_fatures:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

106937


In [343]:
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

In [157]:
max_fatures

107090

In [164]:
from tensorflow.core.protobuf import rewriter_config_pb2
config = tf.ConfigProto()
off = rewriter_config_pb2.RewriterConfig.OFF
config.graph_options.rewrite_options.memory_optimization  = off
return tf.Session(config=config)

#tf.config.optimizer.set_experimental_options({'arithmetic_optimization': False})


AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

In [359]:
from keras.initializers import Constant
embed_dim = 25
lstm_out = 196

model = Sequential()
model.add(Embedding(num_words,
                    embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=X.shape[1],
                    trainable=True))


model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 47, 25)            2673425   
_________________________________________________________________
spatial_dropout1d_11 (Spatia (None, 47, 25)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 392)               348096    
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 786       
Total params: 3,022,307
Trainable params: 3,022,307
Non-trainable params: 0
_________________________________________________________________
None


In [169]:
len(X)

200000

In [322]:
Y = pd.get_dummies(data_new['sentiment']).values
len(Y)

200000

In [363]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(134000, 47) (134000, 2)
(66000, 47) (66000, 2)


In [365]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
4188/4188 - 780s - loss: 0.4947 - accuracy: 0.7403
Epoch 2/7
4188/4188 - 955s - loss: 0.4176 - accuracy: 0.8004
Epoch 3/7
4188/4188 - 940s - loss: 0.3902 - accuracy: 0.8174
Epoch 4/7
4188/4188 - 828s - loss: 0.3732 - accuracy: 0.8282
Epoch 5/7
4188/4188 - 837s - loss: 0.3564 - accuracy: 0.8372
Epoch 6/7
4188/4188 - 803s - loss: 0.3462 - accuracy: 0.8428
Epoch 7/7
4188/4188 - 956s - loss: 0.3341 - accuracy: 0.8495


In [366]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

2016/2016 - 46s - loss: 0.3653 - accuracy: 0.8327
score: 0.37
acc: 0.83


In [367]:
# serialize model to JSON
model_json = model.to_json()
with open("model25dglove.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model25dglove.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
#now predict sentiment in competition data 

In [256]:
test_sentences = []
with open('test_data.txt') as file:
    for line in file:
        tweet= line.split(",")[0]
        tweet = line[len(tweet)+1:]
        test_sentences.append(tweet)



In [257]:
competition_data = pd.DataFrame({"tweet":test_sentences})
competition_data

,tweet
0,sea doo pro sea scooter ( sports with the port...
1,<user> shucks well i work all week so now i ca...
2,i cant stay away from bug thats my baby\n
3,<user> no ma'am ! ! ! lol im perfectly fine an...
4,"whenever i fall asleep watching the tv , i alw..."
...,...
9995,had a nice time w / my friend lastnite\n
9996,<user> no it's not ! please stop !\n
9997,not without my daughter ( dvd two-time oscar (...
9998,<user> have fun in class sweetcheeks\n


In [258]:
#data = data[data.sentiment != "Neutral"]
competition_data['tweet'] = competition_data['tweet'].apply(lambda x: x.lower())
competition_data['tweet'] = competition_data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [266]:
competition_data['tweet']

0       sea doo pro sea scooter  sports with the porta...
1       user shucks well i work all week so now i cant...
2               i cant stay away from bug thats my baby\n
3       user no maam    lol im perfectly fine and not ...
4       whenever i fall asleep watching the tv  i alwa...
                              ...                        
9995              had a nice time w  my friend lastnite\n
9996                      user no its not  please stop \n
9997    not without my daughter  dvd twotime oscar  r ...
9998                 user have fun in class sweetcheeks\n
9999    making a r  e  a  l  difference   get r  e  a ...
Name: tweet, Length: 10000, dtype: object

In [276]:

twts = tokenizer.texts_to_sequences(competition_data['tweet'].values)

twts = pad_sequences(twts, maxlen=47, dtype='int32', value=0)


In [279]:
twts

array([[    0,     0,     0, ...,   812,     8,     6],
       [    0,     0,     0, ...,    14,    28, 10128],
       [    0,     0,     0, ...,    83,     9,   179],
       ...,
       [    0,     0,     0, ...,     4,    80,     6],
       [    0,     0,     0, ...,    14,   355, 79501],
       [    0,     0,     0, ...,    62,   199,     6]], dtype=int32)

In [368]:
preds = model.predict(twts,batch_size=32,verbose = 2)

313/313 - 8s


In [369]:
output_arr=[]
id_count= 1
for pred in preds:
    if(np.argmax(pred ) == 0):
        output_arr.append([id_count, -1])
    elif (np.argmax(pred ) == 1):
        output_arr.append([id_count, 1])
    id_count+=1

In [370]:
output_df = pd.DataFrame(np.array(output_arr))
output_df.columns=["Id", "Prediction"]
output_df.set_index('Id', inplace=True)
output_df.to_csv("Predictions2.csv")

[[9173, 180, 1507, 11, 151, 12, 79, 1811, 79, 29, 11, 151]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0 9173  180 1507   11  151   12   79 1811   79   29   11  151]]
1/1 - 0s
[9.9978906e-01 2.1099388e-04]
negative
